# ReAct vs Function Calling vs Router

ReAct, Function Calling e Router são abordagens diferentes usadas para melhorar as interações com modelos de linguagem, especialmente no contexto de sistemas que exigem maior precisão, estrutura ou capacidade de execução de tarefas específicas. Vamos detalhar cada uma:

### 1. **ReAct (Reasoning and Acting)**
ReAct é uma técnica que combina raciocínio (reasoning) com ações (acting). Aqui, o modelo não apenas gera uma resposta, mas também pode tomar decisões e executar ações com base no raciocínio que faz ao longo do processo de geração de texto. Ele é útil para casos onde é necessário que o modelo tome passos intermediários para chegar a uma resposta final.  
Artigo: https://arxiv.org/pdf/2210.03629

**Exemplo de uso:** Se o modelo precisar resolver um problema de lógica ou realizar cálculos complexos, ele pode "pensar em voz alta" (gerar passos de raciocínio intermediários) e, em seguida, tomar ações apropriadas, como selecionar dados ou fazer cálculos, antes de chegar à resposta final.

### 2. **Function Calling**
O **Function Calling** permite que o modelo de linguagem chame funções específicas durante sua execução. Essa abordagem é útil para integração direta com APIs ou funções do sistema. Quando o modelo identifica que uma determinada tarefa deve ser executada por uma função predefinida (como buscar dados em uma API, realizar cálculos complexos ou manipular dados), ele invoca essa função e utiliza o resultado na sua resposta.  
documentação:
* https://python.langchain.com/v0.2/docs/how_to/function_calling/
* https://python.langchain.com/v0.2/docs/how_to/tool_calling/

**Exemplo de uso:** Se o usuário pedir a previsão do tempo, o modelo pode chamar uma função que consulta uma API de clima, obter os dados necessários e, em seguida, apresentar a previsão ao usuário.

### 3. **Router**
O **Router** refere-se a um sistema ou mecanismo que decide para onde deve ser encaminhada uma determinada solicitação ou tarefa, com base em regras, modelos ou critérios predefinidos. Em vez de o modelo processar diretamente todas as solicitações, o Router pode direcionar cada solicitação para o módulo ou serviço mais apropriado, que pode ser outro modelo, um conjunto de funções específicas ou até um serviço externo.

**Exemplo de uso:** Se um sistema tiver diferentes módulos especializados (por exemplo, um para linguagem natural, outro para consultas a bancos de dados, e outro para cálculos matemáticos), o Router decide qual módulo deve lidar com a solicitação do usuário. Por exemplo, uma consulta matemática seria enviada para o módulo de cálculos, enquanto uma pergunta sobre dados de clientes poderia ser enviada para um módulo de consulta a bancos de dados.

### Comparação e Usos Comuns
- **ReAct** é ideal para tarefas que exigem raciocínio intermediário, como resolução de problemas ou quebra-cabeças, onde o processo de "pensar em voz alta" é importante.
- **Function Calling** é útil quando o modelo precisa interagir diretamente com APIs ou executar funções específicas que requerem dados dinâmicos ou processamento externo.
- **Router** é usado em sistemas complexos com múltiplos módulos, permitindo que as solicitações sejam encaminhadas para os recursos ou serviços mais adequados.

Cada abordagem tem suas vantagens específicas dependendo do contexto e das necessidades do sistema em que estão implementadas.

https://python.langchain.com/v0.2/docs/how_to/routing/


# 1 Agent router
<img src='assets/imgs/agent.bmp' width='600'></img>

In [2]:
from typing import Any

from dotenv import load_dotenv
from langchain import hub
from langchain_core.tools import Tool
from langchain_openai import ChatOpenAI
from langchain.agents import (
    create_react_agent,
    AgentExecutor,
    AgentType
)
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain_experimental.tools import PythonREPLTool
from langchain_experimental.agents.agent_toolkits import create_csv_agent


load_dotenv()

True

Definindo os agentes ou LLMs com suas respectivas tarefas 
 * Temos dois agentes:
   - Executa código python
   - Interação com dados em csv
 * LLM genérico

In [2]:
instructions = """You are an agent designed to write and execute python code to answer questions.
You have access to a python REPL, which you can use to execute python code.
You have qrcode package installed
If you get an error, debug your code and not try again.
Only use the output of your code to answer the question. 
You might know the answer without running any code, but you should still run the code to get the answer.
If it does not seem like you can write code to answer the question, just return "I don't know" as the answer.
    """
    
template_react_agent ="""
    {instructions}

    TOOLS:
    ------

    You have access to the following tools:

    {tools}

    To use a tool, please use the following format:

    ```
    Thought: Do I need to use a tool? Yes
    Action: the action to take, should be one of [{tool_names}]
    Action Input: the input to the action
    Observation: the result of the action
    ```

    When you have a response to say to the Human, or if you do not need to use a tool, you MUST use the format:

    ```
    Thought: Do I need to use a tool? No
    Final Answer: [your response here]
    ```

    Begin!

    Previous conversation history:
    {chat_history}

    New input: {input}
    {agent_scratchpad}
"""
# base_prompt = hub.pull("langchain-ai/react-agent-template")    
base_prompt = PromptTemplate.from_template(template_react_agent)

prompt = base_prompt.partial(instructions=instructions, chat_history='')

tools = [PythonREPLTool()]
python_agent = create_react_agent(
    prompt=prompt,
    llm=ChatOpenAI(temperature=0, model="gpt-4o-mini"),
    tools=tools,
)

python_agent_executor = AgentExecutor(agent=python_agent, tools=tools, verbose=False)

csv_agent_executor: AgentExecutor = create_csv_agent(
    llm=ChatOpenAI(temperature=0, model='gpt-4o-mini'),
    path='data/iris.csv',
    verbose=False,
    # agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    allow_dangerous_code=True,
    
)

generic_chain = ChatOpenAI(temperature=.5, model='gpt-4o-mini')

Necessário envolver os agentes/chain... com a classe Tools. Criamos um agente que séra responsavél por realizar o direcionamento para a ferramenta adequada 

In [3]:
# ################################ Router Grand Agent ########################################################


def python_agent_executor_wrapper(original_prompt: str) -> dict[str, Any]:
    return python_agent_executor.invoke({"input": original_prompt})

tools = [
    Tool(
        name="Python Agent",
        func=python_agent_executor_wrapper,
        description="""useful when you need to transform natural language to python and execute the python code,
                        returning the results of the code execution
                        DOES NOT ACCEPT CODE AS INPUT""",
    ),
    Tool(
        name="CSV Agent",
        func=csv_agent_executor.invoke,
        description="""useful when you need to answer question over iris.csv file,
                        takes an input the entire question and returns the answer after running pandas calculations""",
    ),
     Tool(
        name="Generic chain",
        func=generic_chain.invoke,
        description="""use it when you want an answer that doesn't fit into the other tools. It is a generic tool""",
    ),
]

prompt = base_prompt.partial(instructions="", chat_history='')

grand_agent = create_react_agent(
    prompt=prompt,
    llm=ChatOpenAI(temperature=0, model='gpt-4o-mini'),
    tools=tools
)

grand_agent_executor = AgentExecutor(agent=grand_agent, tools=tools)  # faz o papel do router


In [4]:
grand_agent_executor.invoke(
        {
            "input": "Conte uma piada",
        }
    )

{'input': 'Conte uma piada',
 'output': 'Por que o livro de matemática se suicidou? Porque ele tinha muitos problemas!\n```'}

In [18]:
grand_agent_executor.invoke(
    {
        "input": "Qual a média de comprimento das flores?",
    }
)



> Entering new AgentExecutor chain...
Thought: Para calcular a média de comprimento das flores, preciso calcular a média da coluna `petal.length` do dataframe `df`. Vou fazer isso agora.
Action: python_repl_ast
Action Input: df['petal.length'].mean()3.7580000000000005I now know the final answer
Final Answer: A média de comprimento das flores é aproximadamente 3.76.

> Finished chain.


{'input': 'Qual a média de comprimento das flores?',
 'output': 'A média de comprimento das flores é aproximadamente 3.76.\n```'}

In [ ]:
grand_agent_executor.invoke(
    {
        "input": "Generate and save in current working directory 15 qrcodes that point to `https://github.com/silva-fabiofreitas/`",
    }
)

# 2 Function calling
Definindo as ferramentas ou funções.  
**Observação:** ao usar bind_tools o nome da função deve ter "_"  

<img src='assets/imgs/toolCalling.bmp' width=800></img>




#

In [24]:
def python_agent_executor_wrapper(original_prompt: str) -> dict[str, Any]:
    return python_agent_executor.invoke({"input": original_prompt})

tools = [
    Tool(
        name="Python_Agent",
        func=python_agent_executor_wrapper,
        description="""useful when you need to transform natural language to python and execute the python code,
                        returning the results of the code execution
                        DOES NOT ACCEPT CODE AS INPUT""",
    ),
    Tool(
        name="CSV_Agent",
        func=csv_agent_executor.invoke,
        description="""useful when you need to answer question over iris.csv file,
                        takes an input the entire question and returns the answer after running pandas calculations""",
    ),
     Tool(
        name="Generic_chain",
        func=generic_chain.invoke,
        description="""use it when you want an answer that doesn't fit into the other tools. It is a generic tool""",
    ),
]

# Outra forma de declarar as ferramentas 

from langchain_core.pydantic_v1 import BaseModel, Field

class Python_Agent(BaseModel):
    """useful when you need to transform natural language to python and execute the python code,
       returning the results of the code execution
       DOES NOT ACCEPT CODE AS INPUT."""

    input: str = Field(..., description="Text")

class CSV_Agent(BaseModel):
    """useful when you need to answer question over iris.csv file,
        takes an input the entire question and returns the answer after running pandas calculations"""

    input: str = Field(..., description="Text")

class Generic_chain(BaseModel):
    """use it when you want an answer that doesn't fit into the other tools. It is a generic tool"""

    input: str = Field(..., description="Text")
    
tools_pydantic = [Python_Agent, CSV_Agent, Generic_chain]


In [13]:

from langchain_core.pydantic_v1 import BaseModel, Field

class Python_Agent(BaseModel):
    """useful when you need to transform natural language to python and execute the python code,
       returning the results of the code execution
       DOES NOT ACCEPT CODE AS INPUT."""

    input: str = Field(..., description="Text")

class CSV_Agent(BaseModel):
    """useful when you need to answer question over iris.csv file,
        takes an input the entire question and returns the answer after running pandas calculations"""

    input: str = Field(..., description="Text")

class Generic_chain(BaseModel):
    """use it when you want an answer that doesn't fit into the other tools. It is a generic tool"""

    input: str = Field(..., description="Text")
    
tools_pydantic = [Python_Agent, CSV_Agent, Generic_chain]

In [56]:
from langchain_core.messages import HumanMessage

llm = ChatOpenAI(model="gpt-4o-mini")

llm_with_tools = llm.bind_tools(tools)

query = "Qual a média de comprimento das flores?"
# query = "Conte uma piada?"

messages = [HumanMessage(query)]

# LLM escolhe a função a ser usada
ai_msg = llm_with_tools.invoke(messages)

print(ai_msg.tool_calls)

messages.append(ai_msg)

[{'name': 'CSV_Agent', 'args': {'args': ['Qual a média de comprimento das flores?'], 'config': {'tags': []}}, 'id': 'call_FhjJzzlcW1yRAhxzg6AqBzMS', 'type': 'tool_call'}]


## 2.1 Parse output

<img src="assets/imgs/toolCallingParseOutput.bmp">

In [55]:
for tool_call in ai_msg.tool_calls:
    selected_tool = {"Python_Agent": python_agent_executor_wrapper, "CSV_Agent": csv_agent_executor.invoke, 'Generic_chain':generic_chain.invoke}[tool_call["name"]]
    tool_msg = selected_tool({'input':tool_call})
    messages.append(tool_msg)

messages

[HumanMessage(content='Conte uma piada?'),
 AIMessage(content='Por que o livro de matemática se suicidou?\n\nPorque tinha muitos problemas!', response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 284, 'total_tokens': 301}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f33667828e', 'finish_reason': 'stop', 'logprobs': None}, id='run-46057c20-7d2f-4f29-9173-df7cffc2ca58-0', usage_metadata={'input_tokens': 284, 'output_tokens': 17, 'total_tokens': 301})]

# 3 Router

In [15]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate

llm = ChatOpenAI(model="gpt-4o-mini")

chain = (
    PromptTemplate.from_template(
        """Given the user question below, classify it as either being about `python_code`, `query`, or `Other`.
        
        python_code: useful when you need to transform natural language to python and execute the python code.
        
        query: useful when you need to answer question over iris.csv file,takes an input the entire question and returns the answer after running pandas calculations.
        
        Other: use it when you want an answer that doesn't fit into the other tools. It is a generic tool.

Do not respond with more than one word.

<question>
{input}
</question>

Classification:"""
    )
    | llm
    | StrOutputParser()
)

chain.invoke({"input": "Code python"})

'python_code'

In [16]:
generic_chain = PromptTemplate.from_template(
    """Respond to the following question:

Question: {input}
Answer:"""
) | ChatOpenAI(temperature=.5, model='gpt-4o-mini')


def route(info):
    if "python_code" in info["topic"].lower():
        return python_agent_executor_wrapper
    elif "query" in info["topic"].lower():
        return csv_agent_executor
    else:
        return generic_chain

In [17]:
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

full_chain = {'topic': chain, 'input':lambda x: x['input']} | RunnableLambda(route)

In [19]:
query = "Qual a média de comprimento das flores?"

full_chain.invoke({'input':query})

{'topic': 'query',
 'input': 'Qual a média de comprimento das flores?',
 'output': 'A média de comprimento das flores é aproximadamente 3.76.'}

In [20]:
query = "Conte uma piada?"

full_chain.invoke({'input':query})
# generic_chain.invoke(query)

AIMessage(content='Claro! Aqui vai uma:\n\nPor que o livro de matemática se suicidou?\n\nPorque tinha muitos problemas! 😄', response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 22, 'total_tokens': 46}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f33667828e', 'finish_reason': 'stop', 'logprobs': None}, id='run-468b4694-d9a6-4a1e-b430-9552e54d38fc-0', usage_metadata={'input_tokens': 22, 'output_tokens': 24, 'total_tokens': 46})